In [172]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import random
import requests
import re
import os
import urllib.request

# List of user agents to rotate
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    # Add more user agents as needed
]


In [173]:
# Function to get HTML content from a URL
def get_html_content(url):
    headers = {'User-Agent': random.choice(USER_AGENTS)}  # Randomly select user agent

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to fetch URL: {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [174]:
# Function to extract timestamp and content from HTML
def extract_course_content(url):
    name = re.search(r'courses/(.*?)/', url).group(1)
    
    html_content = get_html_content(url)
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        videos = soup.find_all('a', class_='resource-thumbnail')
        titles = soup.find_all('a', class_='resource-list-title')
        
        videos_txt = f"./{name}/{'videos_'}{name}.txt"
        titles_txt = f"./{name}/{'titles_'}{name}.txt"
        
        with open(videos_txt, 'w') as txt_file:
            for i in range(0, len(videos)):
                txt_file.write(f"{videos[i]['href']}\n")
                
        with open(titles_txt, 'w') as txt_file:
            for i in range(0, len(titles)):
                txt_file.write(f"{'https://ocw.mit.edu/'}{titles[i]['href']}\n")
                
        print(f"Data extracted and written successfully")
    else:
        print("Failed to extract HTML content.")
        
    return (f"./{name}/{'videos_'}{name}.txt", f"./{name}/{'titles_'}{name}.txt")

In [175]:
def get_video_transcript(url, location):
    chrome_options = Options()
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')
    driver = webdriver.Chrome()
    driver.get(url)
    
    driver.find_element(By.CLASS_NAME, value="tab-title-section").click()

    # Wait for some time for the page to load or elements to appear
    time.sleep(5)  # Wait for 5 seconds (adjust as needed)

    # Get HTML content after operations have been performed
    html_content = driver.page_source

    # # Extract timestamp and content from HTML
    # extract_timestamp_content(html_content, url)

    # Close the WebDriver
    driver.quit()
    
    # Use regular expression to extract the desired part
    name = url.split("/")[-2]
    
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        transcript_lines = soup.find_all('div', class_='transcript-line')
        filename = f"./{location}/{'transcripts'}/{name}.txt"
        with open(filename, 'w') as txt_file:
            for line in transcript_lines:
                timestamp = line.find('span', class_='transcript-timestamp').text.strip()
                content = line.find('span', class_='transcript-text').text.strip()
                content = " ".join(content.split("\n"))  # Remove newline characters
                txt_file.write(f"{timestamp} | {content}\n")
        print(f"Data extracted and written to {filename}")
    else:
        print("Failed to extract HTML content.")

In [176]:
def agent(course_url):
    course_name = re.search(r'courses/(.*?)/', course_url).group(1)
    
        # Check if the directory already exists
    if not os.path.exists(course_name):
        # Create the directory
        os.makedirs(course_name)
        os.makedirs(f"{course_name}/{'transcripts'}")
        os.makedirs(f"{course_name}/{'videos'}")
        print("Directory created successfully!")
    else:
        print("Directory already exists!")
        
    raw_data = extract_course_content(course_url)
    
    videos = []
    contents = []
    
    with open(raw_data[0], 'r') as file:
        for line in file:
            videos.append(line.strip())
    
    with open(raw_data[1], 'r') as file:
        for line in file:
            contents.append(line.strip())
            
    for i in range(0, len(videos)):
        get_video_transcript(contents[i], course_name) 
        # name = contents[i].split("/")[-2]
        # urllib.request.urlretrieve(videos[i], f"{course_name}/{'videos'}/{name}.mp4") 

course_url = "https://ocw.mit.edu/courses/6-00-introduction-to-computer-science-and-programming-fall-2008/resources/lecture-videos/"
agent(course_url)
    

Directory created successfully!
Data extracted and written successfully
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-1.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-2.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-3.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-4.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-5.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-6.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-programming-fall-2008/transcripts/lecture-7.txt
Data extracted and written to ./6-00-introduction-to-computer-science-and-